In [13]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation

import bp
import numpy as np
np.set_printoptions(suppress=True)

In [8]:
U = bp.variableNode("Unblocked FEV1", 2, 6, 0.1)
B = bp.variableNode("Small airway clearance", 0, 1, 0.1)
FEV1 = bp.variableNode("FEV1", 0, 6, 0.1)

In [9]:
graph = BayesianNetwork([(U.name, FEV1.name), (B.name, FEV1.name)])

cpt_fev1 = TabularCPD(
    variable=FEV1.name,
    variable_card=len(FEV1.bins)-1,
    values=bp.calc_pgmpy_cpt(U, B, FEV1).tolist(),
    evidence=[U.name, B.name],
    evidence_card=[len(U.bins)-1, len(B.bins)-1],
)

prior_b = TabularCPD(
    variable=B.name,
    variable_card=len(B.bins)-1,
    values=B.marginal(B),
    evidence=[],
    evidence_card=[],
)

prior_u = TabularCPD(
    variable=U.name,
    variable_card=len(U.bins)-1,
    values=U.marginal(U),
    evidence=[],
    evidence_card=[],
)

graph.add_cpds(cpt_fev1, prior_b, prior_u)

graph.check_model()


calculating cpt of shape 60 x 40 x 10 (C x A x B) 


True

In [15]:
cpt = bp.calc_pgmpy_cpt(U, B, FEV1)

calculating cpt of shape 60 x 40 x 10 (C x A x B) 


In [10]:
bp = BeliefPropagation(graph)
res = bp.query(variables=[U.name], evidence={FEV1.name: 3})
print(f"shape of the response {res.values.shape}")
res.values

  0%|          | 0/1 [00:00<?, ?it/s]

shape of the response (40,)


array([0.        , 0.12466071, 0.20311401, 0.16595623, 0.36907025,
       0.1371988 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [11]:
bp = BeliefPropagation(graph)
res = bp.query(variables=[B.name], evidence={FEV1.name: 3})
print(f"shape of the response {res.values.shape}")
res.values
# very uniform

  0%|          | 0/1 [00:00<?, ?it/s]

shape of the response (10,)


array([0.11475862, 0.11085289, 0.1072281 , 0.10385304, 0.10070115,
       0.09774968, 0.09497899, 0.09237203, 0.08991391, 0.08759158])

In [18]:
cpt = bp.calc_pgmpy_cpt(U, B, FEV1)

calculating cpt of shape 60 x 40 x 10 (C x A x B) 


In [20]:
cpt.shape

(60, 400)

In [23]:
cpt[3, :]
# fix unblocked fev1 and fev1
# continue

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01630532, 0.04753905, 0.07685024, 0.10441111, 0.13037389,
       0.15487369, 0.17803077, 0.19995259, 0.22073541, 0.24046576,
       0.24692613, 0.24097552, 0.23530497, 0.22989518, 0.22472856,
       0.21978907, 0.21506205, 0.21053409, 0.20619287, 0.20202707,
       0.19802627, 0.19418086, 0.19048195, 0.18692133, 0.18349139,
       0.18018506, 0.17699577, 0.17391743, 0.17094433, 0.16807118,
       0.48790164, 0.46520016, 0.44451763, 0.42559614, 0.40821995,
       0.39220713, 0.37740328, 0.36367644, 0.3509132 , 0.33901552,
       0.31159291, 0.26994793, 0.23086635, 0.19411853, 0.15950147,
       0.12683508, 0.09595897, 0.06672988, 0.03901946, 0.01271232,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     